## Proof of Concept
The output is the df that gets generated, to be then made into a csv. I've set it up to 10... but it can be set to something larger. 

In [1]:
import pandas as pd

import requests
import json
import pandas as pd

cocktail_id = [] 
names = []
ingredients = [] 
num_ingredients = []  
alcoholic = []
acidic = []

url = "https://www.thecocktaildb.com/api/json/v1/1/random.php"

# Generate 1 random cocktail that is valid
while len(cocktail_id) < 10:
    r = requests.get(url)

    if r.status_code == 200:
        site_json = r.json()

        # Check if the response contains valid data
        if 'drinks' in site_json and site_json['drinks']:
            drink = site_json['drinks'][0]
            name = drink.get('strDrink')

            # Append the columns 
            cocktail_id.append(drink['idDrink'])
            names.append(name)

            # Extract ingredients (removing Nones) -- special case
            # Keeping all arrays equal length; joining all ingredients into a single comma-separated list 
            ingredients_raw = [drink.get(f'strIngredient{i}', '') for i in range(1, 16)]
            ingredients.append(', '.join(filter(None, ingredients_raw)))

            # Calculate and append the number of ingredients
            num_ingredients.append(len(list(filter(None, ingredients_raw))))

            alcoholic_status = drink.get('strAlcoholic', '').lower() == 'alcoholic'
            alcoholic.append(alcoholic_status)

            acidic_status = any(ingredient in ["Lemon", "Lime", "Orange"] for ingredient in ingredients_raw)
            acidic.append(acidic_status)

        else:
            continue

# Create DataFrame
df_cocktail = pd.DataFrame({"cocktail_id": cocktail_id, "cocktail_name": names, "num_ingredients": num_ingredients, "ingredients":ingredients, "alcoholic":alcoholic, "acidic":acidic})
df_cocktail.set_index("cocktail_id", inplace=True)

df_cocktail


,cocktail_name,num_ingredients,ingredients,alcoholic,acidic
cocktail_id,,,,,
16031,Radler,2,"Beer, 7-Up",True,False
12130,Scooter,3,"Brandy, Amaretto, Light cream",True,False
15521,Swedish Coffee,3,"Coffee, Aquavit, Sugar",True,False
13202,Flaming Dr. Pepper,5,"Amaretto, Vodka, 151 proof rum, Dr. Pepper, Beer",True,False
15849,Apricot punch,5,"Apricot brandy, Champagne, Vodka, 7-Up, Orange...",True,False
16176,Bellini Martini,5,"Ice, Vodka, Peach nectar, Peach schnapps, Lemo...",True,False
12708,Banana Cantaloupe Smoothie,2,"Cantaloupe, Banana",False,False
13068,Brave Bull Shooter,2,"Tequila, Tabasco sauce",True,False
15567,Adam Sunrise,4,"Vodka, Lemonade, Water, Sugar",True,False


In [9]:
import os
import requests
import json
import pandas as pd

def generate_cocktail_dataframe(num_cocktails):
    cocktail_id = [] 
    names = []
    ingredients = [] 
    num_ingredients = []  
    alcoholic = []
    acidic = []

    url = "https://www.thecocktaildb.com/api/json/v1/1/random.php"

    # Generate random cocktails that are valid
    while len(cocktail_id) < num_cocktails:
        r = requests.get(url)

        if r.status_code == 200:
            site_json = r.json()

            # Check if the response contains valid data
            if 'drinks' in site_json and site_json['drinks']:
                drink = site_json['drinks'][0]
                name = drink.get('strDrink')

                # Append the columns 
                cocktail_id.append(drink['idDrink'])
                names.append(name)

                # Extract ingredients (removing Nones)
                ingredients_raw = [drink.get(f'strIngredient{i}', '') for i in range(1, 16)]
                ingredients.append(', '.join(filter(None, ingredients_raw)))

                # Calculate and append the number of ingredients
                num_ingredients.append(len(list(filter(None, ingredients_raw))))

                # Convert "strAlcoholic" to boolean
                alcoholic_status = drink.get('strAlcoholic', '').lower() == 'alcoholic'
                alcoholic.append(alcoholic_status)

                # Check if any of the specified ingredients are in the list
                acidic_status = any(ingredient in ["Lemon", "Lime", "Orange"] for ingredient in ingredients_raw)
                acidic.append(acidic_status)

            else:
                continue

    # Create DataFrame
    df_cocktail = pd.DataFrame({
        "cocktail_id": cocktail_id,
        "cocktail_name": names,
        "num_ingredients": num_ingredients,
        "ingredients": ingredients,
        "alcoholic": alcoholic,
        "acidic": acidic
    })

    csv_filename = "cocktails_result.csv"

    # Create full file path in the current working directory
    csv_path = os.path.join(os.getcwd(), csv_filename)

    # Save to CSV
    df_cocktail.to_csv(csv_path, index=False)
    print(f"Data saved to {csv_path}")

    return df_cocktail

# Example usage
generate_cocktail_dataframe(100)

Data saved to c:\Users\andre\Desktop\Personal\CocktailDB_API\cocktails_result.csv


,cocktail_id,cocktail_name,num_ingredients,ingredients,alcoholic,acidic
0,11470,Havana Cocktail,3,"Light rum, Pineapple juice, Lemon juice",True,False
1,11149,Boxcar,5,"Gin, Triple sec, Lemon juice, Grenadine, Egg w...",True,False
2,11053,Arise My Love,2,"Champagne, Green Creme de Menthe",True,False
3,12452,Victory Collins,5,"Vodka, Lemon juice, Grape juice, Powdered suga...",True,True
4,12654,Banana Milk Shake,4,"Milk, Orange juice, Sugar syrup, Banana",False,False
...,...,...,...,...,...,...
95,178306,Slippery Nipple,2,"Sambuca, Irish cream",True,False
96,178368,Planter’s Punch,6,"Dark Rum, Orange Juice, Pineapple Juice, Grena...",True,False
97,11416,Gin Smash,6,"Gin, Carbonated water, Sugar, Mint, Orange, Ch...",True,True
98,17194,White Lady,3,"Gin, Triple Sec, Lemon Juice",True,False
